# B2: Class-weighted Random Forest

This notebook evaluates a class-weighted random forest model using AQ-10 and demographic features.

**Purpose:**  
To assess whether incorporating class weights improves robustness and recall for minority classes in screening tasks.


In [2]:
from pathlib import Path
import sys
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from IPython.display import display

In [3]:
# REPRODUCIBILITY
RANDOM_STATE = 42

In [20]:
# PATHS
NOTEBOOK_DIR = Path.cwd()                 # .../notebooks
PROJECT_ROOT = NOTEBOOK_DIR.parent        # repo root

DATA_DIR = PROJECT_ROOT / "data"
SRC_DIR = PROJECT_ROOT / "src"
RESULTS_DIR = PROJECT_ROOT / "results" / "B2"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)


sys.path.insert(0, str(SRC_DIR))

from common_setup import (
    load_arff,
    prepare_df_AQ_DEMO,
    build_preprocessor,
    run_experiment
)

ADULT_ARFF_PATH = DATA_DIR / "Autism-Adult-Data.arff"
CHILD_ARFF_PATH = DATA_DIR / "Autism-Child-Data.arff"

In [5]:
# LOAD DATA (ARFF)

adult_raw = load_arff(ADULT_ARFF_PATH)
child_raw = load_arff(CHILD_ARFF_PATH)

print("\nLoaded adult shape:", adult_raw.shape)
print("Loaded child shape:", child_raw.shape)



Loaded adult shape: (704, 21)
Loaded child shape: (292, 21)


In [8]:
# PREP FEATURES (AQ + DEMO)

Xa, ya, num_adult, cat_adult, feat_adult = prepare_df_AQ_DEMO(adult_raw, "Adult")
Xc, yc, num_child, cat_child, feat_child = prepare_df_AQ_DEMO(child_raw, "Child")



[Adult] AQ columns: ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']
[Adult] Dropping leaky column 'result'
[Adult] Using label column: 'Class/ASD'
[Adult] X shape: (704, 19)
[Adult] y counts: [515 189]

[Child] AQ columns: ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']
[Child] Dropping leaky column 'result'
[Child] Using label column: 'Class/ASD'
[Child] X shape: (292, 19)
[Child] y counts: [151 141]


In [12]:
# 4) BUILD PIPELINES
#    Random Forest needs one-hot encoding for categorical features
# ------------------------------------------------
pre_adult = build_preprocessor(num_adult, cat_adult)
pre_child = build_preprocessor(num_child, cat_child)

# Class-weighted RF to handle class imbalance
# min_samples_leaf=2 helps generalization (less overfitting)
b2_adult_clf = RandomForestClassifier(
    n_estimators=500,
    random_state=RANDOM_STATE,
    class_weight="balanced",
    n_jobs=-1,
    min_samples_leaf=2
)

b2_child_clf = RandomForestClassifier(
    n_estimators=500,
    random_state=RANDOM_STATE,
    class_weight="balanced",
    n_jobs=-1,
    min_samples_leaf=2
)

b2_adult_pipe = Pipeline([
    ("prep", pre_adult),
    ("clf", b2_adult_clf)
])

b2_child_pipe = Pipeline([
    ("prep", pre_child),
    ("clf", b2_child_clf)
])

In [14]:
# RUN 5-FOLD CV
# ------------------------------------------------
adult_folds, adult_summary = run_experiment(
    Xa, ya, b2_adult_pipe,
    exp_name="B2 Adult (Class-weighted RF AQ+Demo)",
    n_splits=5, random_state=RANDOM_STATE
)

child_folds, child_summary = run_experiment(
    Xc, yc, b2_child_pipe,
    exp_name="B2 Child (Class-weighted RF AQ+Demo)",
    n_splits=5, random_state=RANDOM_STATE
)


[B2 Adult (Class-weighted RF AQ+Demo)] Fold 1: acc=0.9787, prec=0.9268, rec=1.0000, f1=0.9620, roc_auc=0.9987, pr_auc=0.9966
[B2 Adult (Class-weighted RF AQ+Demo)] Fold 2: acc=0.9645, prec=0.9714, rec=0.8947, f1=0.9315, roc_auc=0.9962, pr_auc=0.9901
[B2 Adult (Class-weighted RF AQ+Demo)] Fold 3: acc=0.9149, prec=0.7826, rec=0.9474, f1=0.8571, roc_auc=0.9859, pr_auc=0.9655
[B2 Adult (Class-weighted RF AQ+Demo)] Fold 4: acc=0.9504, prec=0.9189, rec=0.8947, f1=0.9067, roc_auc=0.9939, pr_auc=0.9852
[B2 Adult (Class-weighted RF AQ+Demo)] Fold 5: acc=0.9643, prec=0.9444, rec=0.9189, f1=0.9315, roc_auc=0.9940, pr_auc=0.9858

[B2 Adult (Class-weighted RF AQ+Demo)] 5-fold mean metrics:
  accuracy: 0.9546
  precision: 0.9088
  recall: 0.9312
  f1: 0.9178
  roc_auc: 0.9937
  pr_auc: 0.9847
[B2 Child (Class-weighted RF AQ+Demo)] Fold 1: acc=0.9153, prec=0.8710, rec=0.9643, f1=0.9153, roc_auc=0.9712, pr_auc=0.9732
[B2 Child (Class-weighted RF AQ+Demo)] Fold 2: acc=0.9492, prec=0.9643, rec=0.9310, f

In [15]:
# SAVE CV OUTPUTS
# ------------------------------------------------
adult_folds.to_csv(RESULTS_DIR / "B2_adult_folds.csv", index=False)
adult_summary.to_csv(RESULTS_DIR / "B2_adult_summary.csv", index=False)

child_folds.to_csv(RESULTS_DIR / "B2_child_folds.csv", index=False)
child_summary.to_csv(RESULTS_DIR / "B2_child_summary.csv", index=False)

print("\nSaved CV results to:")
print(" ", RESULTS_DIR / "B2_adult_folds.csv")
print(" ", RESULTS_DIR / "B2_adult_summary.csv")
print(" ", RESULTS_DIR / "B2_child_folds.csv")
print(" ", RESULTS_DIR / "B2_child_summary.csv")

print("\nB2 Adult mean metrics:")
display(adult_summary)

print("\nB2 Child mean metrics:")
display(child_summary)


Saved CV results to:
  C:\Users\14ush\Desktop\asc-screening-xai-dissertation\results\B2\B2_adult_folds.csv
  C:\Users\14ush\Desktop\asc-screening-xai-dissertation\results\B2\B2_adult_summary.csv
  C:\Users\14ush\Desktop\asc-screening-xai-dissertation\results\B2\B2_child_folds.csv
  C:\Users\14ush\Desktop\asc-screening-xai-dissertation\results\B2\B2_child_summary.csv

B2 Adult mean metrics:


,accuracy,precision,recall,f1,roc_auc,pr_auc
0,0.954559,0.908846,0.931152,0.91777,0.993734,0.984651



B2 Child mean metrics:


,accuracy,precision,recall,f1,roc_auc,pr_auc
0,0.931502,0.913165,0.950493,0.930763,0.985956,0.986458


In [16]:
# SHUFFLED-LABEL SANITY CHECK

print("\nRunning shuffled-label sanity check (expect performance ~ random)...")

def shuffled_label_run(X, y, pipe, name, random_state=RANDOM_STATE):
    y_shuf = shuffle(y, random_state=random_state)
    shuf_folds, shuf_summary = run_experiment(
        X, y_shuf, pipe,
        exp_name=f"{name} (SHUFFLED LABELS)",
        n_splits=5, random_state=random_state
    )
    return shuf_folds, shuf_summary

adult_shuf_folds, adult_shuf_summary = shuffled_label_run(Xa, ya, b2_adult_pipe, "B2 Adult")
child_shuf_folds, child_shuf_summary = shuffled_label_run(Xc, yc, b2_child_pipe, "B2 Child")

adult_shuf_folds.to_csv(RESULTS_DIR / "B2_adult_shuffled_folds.csv", index=False)
adult_shuf_summary.to_csv(RESULTS_DIR / "B2_adult_shuffled_summary.csv", index=False)

child_shuf_folds.to_csv(RESULTS_DIR / "B2_child_shuffled_folds.csv", index=False)
child_shuf_summary.to_csv(RESULTS_DIR / "B2_child_shuffled_summary.csv", index=False)

print("\nSaved shuffled-label results to:")
print(" ", RESULTS_DIR / "B2_adult_shuffled_folds.csv")
print(" ", RESULTS_DIR / "B2_adult_shuffled_summary.csv")
print(" ", RESULTS_DIR / "B2_child_shuffled_folds.csv")
print(" ", RESULTS_DIR / "B2_child_shuffled_summary.csv")


print("\nB2 Adult shuffled-label mean metrics:")
display(adult_shuf_summary)

print("\nB2 Child shuffled-label mean metrics:")
display(child_shuf_summary)



Running shuffled-label sanity check (expect performance ~ random)...
[B2 Adult (SHUFFLED LABELS)] Fold 1: acc=0.6312, prec=0.1500, rec=0.0789, f1=0.1034, roc_auc=0.5263, pr_auc=0.2732
[B2 Adult (SHUFFLED LABELS)] Fold 2: acc=0.6454, prec=0.2273, rec=0.1316, f1=0.1667, roc_auc=0.4790, pr_auc=0.2631
[B2 Adult (SHUFFLED LABELS)] Fold 3: acc=0.6596, prec=0.3077, rec=0.2105, f1=0.2500, roc_auc=0.5227, pr_auc=0.2763
[B2 Adult (SHUFFLED LABELS)] Fold 4: acc=0.6099, prec=0.1304, rec=0.0789, f1=0.0984, roc_auc=0.4885, pr_auc=0.2632
[B2 Adult (SHUFFLED LABELS)] Fold 5: acc=0.6929, prec=0.3125, rec=0.1351, f1=0.1887, roc_auc=0.5484, pr_auc=0.3278

[B2 Adult (SHUFFLED LABELS)] 5-fold mean metrics:
  accuracy: 0.6478
  precision: 0.2256
  recall: 0.1270
  f1: 0.1614
  roc_auc: 0.5130
  pr_auc: 0.2807
[B2 Child (SHUFFLED LABELS)] Fold 1: acc=0.5085, prec=0.4783, rec=0.3929, f1=0.4314, roc_auc=0.4896, pr_auc=0.5345
[B2 Child (SHUFFLED LABELS)] Fold 2: acc=0.5763, prec=0.5667, rec=0.5862, f1=0.5763, 

,accuracy,precision,recall,f1,roc_auc,pr_auc
0,0.647791,0.22558,0.127027,0.161431,0.513004,0.280707



B2 Child shuffled-label mean metrics:


,accuracy,precision,recall,f1,roc_auc,pr_auc
0,0.547984,0.528641,0.524384,0.524941,0.559113,0.558588
